<a href="https://colab.research.google.com/github/aknip/Modal/blob/main/iCloud_Access.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# iCloud Access


see https://github.com/picklepete/pyicloud


In [ ]:
import json
import os
from getpass import getpass
import psutil
IN_NOTEBOOK = any(["jupyter-notebook" in i for i in psutil.Process().parent().cmdline()])
if IN_NOTEBOOK:
  CREDS = json.loads(getpass("Secrets (JSON string): "))
  os.environ['CREDS'] = json.dumps(CREDS)
  CREDS = json.loads(os.getenv('CREDS'))

In [ ]:
test = CREDS['xxx']['v1']['credential']

In [ ]:
%%capture --no-stderr
!pip install modal nbdev pyicloud dropbox -q

# iCloud

- change email and pw in line 2
- works and tested with 2-factor-auth

In [ ]:
from pyicloud import PyiCloudService
api = PyiCloudService('name', 'pw')
if api.requires_2fa:
    print("Two-factor authentication required.")
    code = input("Enter the code you received of one of your approved devices: ")
    result = api.validate_2fa_code(code)
    print("Code validation result: %s" % result)

    if not result:
        print("Failed to verify security code")
        sys.exit(1)

    if not api.is_trusted_session:
        print("Session is not trusted. Requesting trust...")
        result = api.trust_session()
        print("Session trust result %s" % result)

        if not result:
            print("Failed to request trust. You will likely be prompted for the code again in the coming weeks")
elif api.requires_2sa:
    import click
    print("Two-step authentication required. Your trusted devices are:")

    devices = api.trusted_devices
    for i, device in enumerate(devices):
        print(
            "  %s: %s" % (i, device.get('deviceName',
            "SMS to %s" % device.get('phoneNumber')))
        )

    device = click.prompt('Which device would you like to use?', default=0)
    device = devices[device]
    if not api.send_verification_code(device):
        print("Failed to send verification code")
        sys.exit(1)

    code = click.prompt('Please enter validation code')
    if not api.validate_verification_code(device, code):
        print("Failed to verify verification code")
        sys.exit(1)

In [ ]:
# this does not work? Throws error
# test = api.files()

In [ ]:
# this works!
my_devices = api.devices
my_dir = api.drive.dir()
print(my_devices)
print(my_dir)

In [ ]:
# this works!
drive_file = api.drive['Calibre Library']['Alfons Schmid']['Notebooks App - Handbook (2678)']['Notebooks App - Handbook - Alfons Schmid.pdf']
print(drive_file.date_modified)
# copy to local dir
from shutil import copyfileobj
with drive_file.open(stream=True) as response:
    with open(drive_file.name, 'wb') as file_out:
        copyfileobj(response.raw, file_out)

# Dropbox

- Example from SDK:
  - https://github.com/dropbox/dropbox-sdk-python/tree/main/example/back-up-and-restore
- Alternatives / additional frameworks:
  - https://pycad.co/dropbox-automations/

In [ ]:
# Source: https://practicaldatascience.co.uk/data-science/how-to-use-the-dropbox-api-with-python

import pathlib
import pandas as pd
import dropbox
from dropbox.exceptions import AuthError
DROPBOX_ACCESS_TOKEN = ''

def dropbox_connect():
    """Create a connection to Dropbox."""

    try:
        dbx = dropbox.Dropbox(DROPBOX_ACCESS_TOKEN)
    except AuthError as e:
        print('Error connecting to Dropbox with access token: ' + str(e))
    return dbx

def dropbox_list_files(path):
    #Return a Pandas dataframe of files in a given Dropbox folder path in the Apps directory.
    dbx = dropbox_connect()
    try:
        files = dbx.files_list_folder(path).entries
        files_list = []
        for file in files:
            if isinstance(file, dropbox.files.FileMetadata):
                metadata = {
                    'name': file.name,
                    #'path_display': file.path_display,
                    #'client_modified': file.client_modified,
                    'server_modified': file.server_modified
                }
                files_list.append(metadata)

        df = pd.DataFrame.from_records(files_list)
        #return df
        return df.sort_values(by='server_modified', ascending=False)

    except Exception as e:
        print('Error getting list of files from Dropbox: ' + str(e))

def dropbox_list_files_recursively(path):
    #Return a Pandas dataframe of files in a given Dropbox folder path in the Apps directory.
    dbx = dropbox_connect()
    try:
        files = dbx.files_list_folder(path, recursive=True)
        files_entries = files.entries

        # statt .path_display alternativ auch .name
        list_of_folders = [x.path_display for x in files_entries if 'size' not in dir(x)]
        list_of_files_and_folders = [x.path_display for x in files_entries]
        return list_of_folders, list_of_files_and_folders

    except Exception as e:
        print('Error getting list of files from Dropbox: ' + str(e))

In [ ]:
result = dropbox_list_files_recursively('')
print(json.dumps(result[0], sort_keys=True, indent=2))
print()
print(json.dumps(result[1], sort_keys=True, indent=2))

In [ ]:
result = dropbox_list_files('/boox pdfs/Readme/')
print(result)